In [11]:
from azureml.core import Dataset
from azureml.core import Workspace
import datetime

ws = Workspace.from_config()
dataset = Dataset.get_by_name(ws, name='btc-raw')

df = dataset.to_pandas_dataframe()
df = df.rename(columns={"Column1": "dateid", "Column2": "price","Column3":"volume"})
df

,dateid,price,volume
0,1426616869,290.00,3.045430
1,1426685492,290.00,0.042517
2,1426759978,290.00,2.521450
3,1426760403,290.00,0.625478
4,1426760423,290.00,4.521450
...,...,...,...
198528,1532728105,7049.04,0.001698
198529,1532728155,7049.04,0.200000
198530,1532728155,7049.05,0.001698
198531,1532728156,7049.05,0.198302


In [12]:
dateparse = lambda x: datetime.datetime.utcfromtimestamp(float(x))
df['date'] = [dateparse(x) for x in df['dateid']]
df = df.set_index('date')
del df['dateid']
df['ask'] = df['price']*1.01
df['bid'] = df['price']*0.99
df = df.reindex(columns=['price','ask','bid','volume'])
df

,price,ask,bid,volume
date,,,,
2015-03-17 18:27:49,290.00,292.9000,287.1000,3.045430
2015-03-18 13:31:32,290.00,292.9000,287.1000,0.042517
2015-03-19 10:12:58,290.00,292.9000,287.1000,2.521450
2015-03-19 10:20:03,290.00,292.9000,287.1000,0.625478
2015-03-19 10:20:23,290.00,292.9000,287.1000,4.521450
...,...,...,...,...
2018-07-27 21:48:25,7049.04,7119.5304,6978.5496,0.001698
2018-07-27 21:49:15,7049.04,7119.5304,6978.5496,0.200000
2018-07-27 21:49:15,7049.05,7119.5405,6978.5595,0.001698


In [14]:
#df['prev'] = df['price'].rolling(2).apply(lambda x: (x[1]/x[0])-1 ,raw=True)
    
#df['ma100']=df['price']/df['price'].rolling(10).mean()-1
df['ma10001']=df['price']/df['price'].rolling('60s').mean()-1
#df['ma10002']=df['price']/df['price'].rolling('60s').max()-1
#df['ma10003']=df['price']/df['price'].rolling('60s').min()-1

df['ma1000x1']=df['price']/df['price'].rolling('1h').mean()-1
df['ma1000x2']=df['price']/df['price'].rolling('1h').max()-1
df['ma1000x3']=df['price']/df['price'].rolling('1h').min()-1

df['ma1000y1']=df['price']/df['price'].rolling('10h').mean()-1
#df['ma1000y2']=df['price']/df['price'].rolling('10h').max()-1
#df['ma1000y3']=df['price']/df['price'].rolling('10h').min()-1

df['ma100001']=df['price']/df['price'].rolling('1D').mean()-1
#df['ma100002']=df['price']/df['price'].rolling('1D').max()-1
#df['ma100003']=df['price']/df['price'].rolling('1D').min()-1
df['ma1']=df['price']/df['price'].rolling('10D').mean()-1
#df['ma2']=df['price']/df['price'].rolling('10D').max()-1
#df['ma3']=df['price']/df['price'].rolling('10D').min()-1
df['ma4'] = df['ma1'] / df['price'].rolling('10D').std()
df['mafff1']=df['price']/df['price'].rolling('20D').mean()-1
df['mafff2']=df['price']/df['price'].rolling('20D').max()-1
df['mafff3']=df['price']/df['price'].rolling('20D').min()-1
df['mafff4'] = df['mafff1'] / df['price'].rolling('20D').std()
df = df.dropna()
df

,price,ask,bid,volume,ma10001,ma1000x1,ma1000x2,ma1000x3,ma1000y1,ma100001,ma1,ma4,mafff1,mafff2,mafff3,mafff4
date,,,,,,,,,,,,,,,,
2015-03-20 09:34:51,270.00,272.7000,267.3000,0.062515,0.000000e+00,0.000000,0.0,0.000000,0.000000,-0.061776,-0.063091,-0.010463,-0.063091,-0.068966,0.000000,-0.010463
2015-03-20 14:24:27,270.00,272.7000,267.3000,0.142156,0.000000e+00,0.000000,0.0,0.000000,0.000000,0.000000,-0.058140,-0.007468,-0.058140,-0.068966,0.000000,-0.007468
2015-03-20 16:27:46,270.00,272.7000,267.3000,0.483361,0.000000e+00,0.000000,0.0,0.000000,0.000000,0.000000,-0.053908,-0.006146,-0.053908,-0.068966,0.000000,-0.006146
2015-03-21 22:42:41,270.00,272.7000,267.3000,0.250000,0.000000e+00,0.000000,0.0,0.000000,0.000000,0.000000,-0.050251,-0.005359,-0.050251,-0.068966,0.000000,-0.005359
2015-03-22 16:43:15,270.00,272.7000,267.3000,0.173080,0.000000e+00,0.000000,0.0,0.000000,0.000000,0.000000,-0.047059,-0.004822,-0.047059,-0.068966,0.000000,-0.004822
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018-07-27 21:48:25,7049.04,7119.5304,6978.5496,0.001698,-1.797451e-13,0.000755,0.0,0.001276,0.020358,0.024152,0.069627,0.000218,0.155064,-0.035374,0.409805,0.000263
2018-07-27 21:49:15,7049.04,7119.5304,6978.5496,0.200000,-2.695622e-13,0.000730,0.0,0.001276,0.020288,0.024101,0.069614,0.000217,0.155046,-0.035374,0.409805,0.000263
2018-07-27 21:49:15,7049.05,7119.5405,6978.5595,0.001698,9.457546e-07,0.000707,0.0,0.001277,0.020221,0.024051,0.069602,0.000217,0.155030,-0.035373,0.409807,0.000263
